# Problem Statement: Using Unsupervised Learning to Sort Service Categories

### Import Libraries

In [2]:
import re
import pandas as pd
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt

from progressbar import ProgressBar

pd.set_option("display.max_columns",500)
pd.set_option('display.max_colwidth', -1)

### Import Data

In [3]:
df2009 = pd.read_csv('./data/City_Service_Requests_in_2009.csv')
df2010 = pd.read_csv('./data/City_Service_Requests_in_2010.csv')
df2011 = pd.read_csv('./data/City_Service_Requests_in_2011.csv')
df2012 = pd.read_csv('./data/City_Service_Requests_in_2012.csv')
df2013 = pd.read_csv('./data/City_Service_Requests_in_2013.csv')
df2014 = pd.read_csv('./data/City_Service_Requests_in_2014.csv')
df2015 = pd.read_csv('./data/City_Service_Requests_in_2015.csv')
df2016 = pd.read_csv('./data/City_Service_Requests_in_2016.csv')
df2017 = pd.read_csv('./data/City_Service_Requests_in_2017.csv')
df2018 = pd.read_csv('./data/City_Service_Requests_in_2018.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#create main df
df = pd.DataFrame(df2009)

In [5]:
#append all years together
df = df.append([df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018])

In [6]:
#drop dup/irrev columns
df.drop(['X', 'Y', 'SERVICECODE', 'SERVICECALLCOUNT', 'XCOORD', 'YCOORD', 'SERVICEORDERSTATUS', 
         'SERVICEREQUESTID', 'CITY', 'STATE', 'MARADDRESSREPOSITORYID'], axis=1, inplace=True)

In [7]:
#how many entries are in the total df
df.shape

(2032019, 19)

In [8]:
#need to reindex so that all of the dfs don't have duplicate index values
df.reset_index(inplace=True, drop=True)

# Data Dictionary

- OBJECTID: claim id number
- SERVICECODEDESCRIPTION: brief text description of service requested
- SERVICETYPECODEDESCRIPTION: coded service
- ORGANIZATIONACRONYM: organization responsible for this service
- ADDDATE: data added to 311 log
- RESOLUTIONDATE: data that the claim was (presumably) resolved (ticket was closed?)
- SERVICEDUEDATE: date the claim service was due?
- SERVICEORDERDATE: date the claim's service was ordered (pushed to the org docket?)
- INSPECTIONFLAG: ?
- INSPECTIONDATE: date the service was inspected
- INSPECTORNAME: person who did the inspection
- STATUS_CODE: ?
- PRIORITY: ?
- STREETADDRESS: address of claim?
- LATITUDE
- LONGITUDE
- ZIPCODE
- WARD
- DETAILS: text summary of situation.

In [9]:
#are there any nulls?
df.isnull().sum()

OBJECTID                      0      
SERVICECODEDESCRIPTION        0      
SERVICETYPECODEDESCRIPTION    855    
ORGANIZATIONACRONYM           2      
ADDDATE                       0      
RESOLUTIONDATE                159854 
SERVICEDUEDATE                27388  
SERVICEORDERDATE              0      
INSPECTIONFLAG                0      
INSPECTIONDATE                1451445
INSPECTORNAME                 1903391
STATUS_CODE                   622674 
PRIORITY                      2679   
STREETADDRESS                 67584  
LATITUDE                      21851  
LONGITUDE                     21851  
ZIPCODE                       22729  
WARD                          30294  
DETAILS                       978416 
dtype: int64

# Cleaning & Exploring

In [10]:
df.SERVICECODEDESCRIPTION.value_counts()

Parking Meter Repair                                                  428395
Bulk Collection                                                       296316
Parking Enforcement                                                   195040
Streetlight Repair Investigation                                      65795 
Pothole                                                               59250 
Emergency No-Parking Verification                                     55183 
Trash Collection - Missed                                             52282 
Sanitation Enforcement                                                48899 
Alley Cleaning                                                        47424 
Residential Parking Permit Violation                                  37716 
Recycling Collection - Missed                                         33368 
Tree Inspection                                                       32067 
Parking Meter Issues                                                  31523 

In [11]:
#how many different services are there?
df.SERVICECODEDESCRIPTION.value_counts().count()
#there are 316 different descriptions

316

In [12]:
#what are the the ten most frequently asigned service codes?
df.SERVICECODEDESCRIPTION.value_counts(normalize=True).head(10)
#Parking Meter Repair                    0.210822
#Bulk Collection                         0.145823
#Parking Enforcement                     0.095983
#Streetlight Repair Investigation        0.032379
#Pothole                                 0.029158
#Emergency No-Parking Verification       0.027157
#Trash Collection - Missed               0.025729
#Sanitation Enforcement                  0.024064
#Alley Cleaning                          0.023338
#Residential Parking Permit Violation    0.018561

Parking Meter Repair                    0.210822
Bulk Collection                         0.145823
Parking Enforcement                     0.095983
Streetlight Repair Investigation        0.032379
Pothole                                 0.029158
Emergency No-Parking Verification       0.027157
Trash Collection - Missed               0.025729
Sanitation Enforcement                  0.024064
Alley Cleaning                          0.023338
Residential Parking Permit Violation    0.018561
Name: SERVICECODEDESCRIPTION, dtype: float64

In [19]:
#how many versions of "tree trimming" are there?
xmas_rows = [index for index, value in df.SERVICECODEDESCRIPTION.iteritems() if 'Christmas' in value]
df.iloc[xmas_rows].SERVICECODEDESCRIPTION.value_counts()

Christmas Tree Removal-Seasonal      705
Christmas Tree Removal - Seasonal    531
Christmas Tree Removal – Seasonal    1  
Name: SERVICECODEDESCRIPTION, dtype: int64

In [20]:
#are there other versions of "tree"?
tree_rows = [index for index, value in df.SERVICECODEDESCRIPTION.iteritems() if 'Tree' in value]
df.iloc[tree_rows].SERVICECODEDESCRIPTION.value_counts() 
#these needs to be renamed to eliminate typo and doubles

Tree Inspection                      32067
Tree Pruning                         21744
Tree Planting                        19103
Tree Removal                         15715
xxx_Tree Trimming LOOK UP ONLY       2091 
Christmas Tree Removal-Seasonal      705  
Christmas Tree Removal - Seasonal    531  
Emergency - Trees                    460  
xxx_Tree Maintenance LOOK UP ONLY    3    
Christmas Tree Removal – Seasonal    1    
Name: SERVICECODEDESCRIPTION, dtype: int64

In [21]:
#can I clean away the xxx_ so that it doesn't group in unsup learning?
df.SERVICECODEDESCRIPTION = df.SERVICECODEDESCRIPTION.map(lambda x: x.replace('xxx_', ''))
df.SERVICECODEDESCRIPTION = df.SERVICECODEDESCRIPTION.map(lambda x: x.replace(' LOOK UP ONLY', ''))

#how can this be cleaned to aggregate these more effeciently?

#is there an easier way to look at all of these categories to make sense of them?
#use unsup learning to sort SERVICECODEDESCRIPTION 

## Unsupervised Learning to Sort Service Codes

In [22]:
#import countvect to count test frequency
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate a CountVectorizer
vect = CountVectorizer(ngram_range=(1, 1))

#fit the strings
X = vect.fit_transform(df.SERVICECODEDESCRIPTION)

In [23]:
#initialize KMeans
from sklearn.cluster import KMeans

km = KMeans(n_clusters=150, n_jobs=-1)
km.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=150, n_init=10, n_jobs=-1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [24]:
preds = km.labels_
labels = pd.DataFrame({'label_code':preds})
labels['service'] = df.SERVICECODEDESCRIPTION

for i in range(150):
    print(i)
    print(labels[labels.label_code == i].service.value_counts())
    print()

0
Sign Removal Investigation    1181
Sign Removal investigation    658 
Sign Removal                  28  
Name: service, dtype: int64

1
Parking Meter Repair                 428395
Parking Meter - Flashing Fail        43    
Parking Meter - Multispace Repair    33    
Parking Meter - Coin Jam             23    
Parking Meter - Bad Battery          2     
Parking Meter - Physical Damage      1     
Name: service, dtype: int64

2
Bulk Collection                             296316
Bulk Collection - Unscheduled               94    
Bulk Collection - Update Existing Pickup    1     
Name: service, dtype: int64

3
Parking Enforcement        195040
Parking - Handicapped      48    
Parking Signs - Install    35    
Parking Regulations        15    
Name: service, dtype: int64

4
Abandoned Vehicle - On Public Property    26587
Name: service, dtype: int64

5
Streetlight Repair Investigation    65795
Name: service, dtype: int64

6
Insects                                 1302
Eviction           

Resident Parking Permit    1160
Name: service, dtype: int64

95
DMV - Processing Center Manager    666
Name: service, dtype: int64

96
Child Safety Seat Program    555
Pedestrian Safety Program    195
Name: service, dtype: int64

97
Litter Can - Collection    1408
Name: service, dtype: int64

98
Bus/Rail Issues    1080
Name: service, dtype: int64

99
Parks and Recreation    527
Name: service, dtype: int64

100
Street Sweeping    987
Name: service, dtype: int64

101
Utility Repair Issue    1145
Name: service, dtype: int64

102
Christmas Tree Removal-Seasonal      705
Christmas Tree Removal - Seasonal    531
Christmas Tree Removal – Seasonal    1  
Name: service, dtype: int64

103
Curb & Gutter Repair - Major    1225
Name: service, dtype: int64

104
Marking Installation    917
Name: service, dtype: int64

105
Insect Treatment    684
Name: service, dtype: int64

106
DMV - Refunds - Tickets                          625
DMV - Tag Surrenders/Registration Fee Refunds    5  
Name: service, dty

In [25]:
#can I write a function to create a new column that reports out the most frequent response?

#can I make a dictionary that can map the new column?
label_dict = {}
for i in range(150):
    label_dict[i] = str(labels[labels.label_code == i].service.value_counts().index[0])
label_dict

{0: 'Sign Removal Investigation',
 1: 'Parking Meter Repair',
 2: 'Bulk Collection',
 3: 'Parking Enforcement',
 4: 'Abandoned Vehicle - On Public Property',
 5: 'Streetlight Repair Investigation',
 6: 'Insects',
 7: 'Emergency No-Parking Verification',
 8: 'Residential Parking Permit Violation',
 9: 'Tree Removal',
 10: 'Sidewalk Repair',
 11: 'Illegal Dumping',
 12: 'Sanitation Enforcement',
 13: 'Recycling Collection - Missed',
 14: 'Trash Collection - Missed',
 15: 'Pothole',
 16: 'Street Cleaning',
 17: 'DMV - Drivers License/ID Issues',
 18: 'Trash Cart - Delivery',
 19: 'Out of State Parking Violation (ROSA)',
 20: 'Tree Inspection',
 21: 'Roadway Signs',
 22: 'Rodent Inspection and Treatment',
 23: 'Alley Cleaning',
 24: 'Dead Animal Collection',
 25: 'Parking Meter Issues',
 26: 'TRU Report',
 27: 'Yard Waste - Missed',
 28: 'Snow/Ice Removal',
 29: 'CONTAINER REMOVAL',
 30: 'Graffiti Removal',
 31: 'DMV - Vehicle Registration Issues',
 32: 'Traffic Signal Maintenance',
 33: '